In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install datasets

In [6]:
from datasets import load_dataset, Dataset


data = load_dataset("ErnestSDavis/winograd_wsc", "wsc285")

In [7]:
print(data['test'][0])

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.', 'pronoun': 'they', 'pronoun_loc': 63, 'quote': 'they feared violence', 'quote_loc': 63, 'options': ['The city councilmen', 'The demonstrators'], 'label': 0, 'source': '(Winograd 1972)'}


In [8]:
train_dataset = Dataset.from_dict(data['test'][:250])  
test_dataset = Dataset.from_dict(data['test'][250:]) 

In [9]:
print(f'{train_dataset[0]} \n')
print(test_dataset[0])

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.', 'pronoun': 'they', 'pronoun_loc': 63, 'quote': 'they feared violence', 'quote_loc': 63, 'options': ['The city councilmen', 'The demonstrators'], 'label': 0, 'source': '(Winograd 1972)'} 

{'text': 'Dan had to stop Bill from toying with the injured bird. He is very compassionate.', 'pronoun': 'He', 'pronoun_loc': 56, 'quote': 'He is very compassionate.', 'quote_loc': 56, 'options': ['Dan', 'Bill'], 'label': 0, 'source': 'David Bender'}


In [10]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install --upgrade --no-cache-dir transformers

In [11]:
from unsloth import FastLanguageModel
import torch 

max_seq_length = 2048
dtype = None
load_in_4bit = True #using 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [12]:
#Lora adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [13]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [14]:
def format_example(example):
    system_msg = {"role": "system", "content": "You are an AI that resolves pronoun references."}
    user_msg = {
        "role": "user",
        "content": f"{example['text']} Who does '{example['pronoun']}' refer to?"
    }
    assistant_msg = {
        "role": "assistant",
        "content": example['options'][example['label']]
    }
    return {"conversations": [system_msg, user_msg, assistant_msg]}

# Apply formatting to the dataset
formatted_dataset = train_dataset.map(format_example)


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [15]:
from unsloth.chat_templates import standardize_sharegpt

# Standardize the dataset to ShareGPT format
standardized_dataset = standardize_sharegpt(formatted_dataset)

Standardizing format:   0%|          | 0/250 [00:00<?, ? examples/s]

In [16]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

In [17]:
formatted_dataset = standardized_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [18]:
train_dataset[0]

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.',
 'pronoun': 'they',
 'pronoun_loc': 63,
 'quote': 'they feared violence',
 'quote_loc': 63,
 'options': ['The city councilmen', 'The demonstrators'],
 'label': 0,
 'source': '(Winograd 1972)'}

In [19]:


# Function to format each example into a conversation
def format_example(example):
    system_msg = {"role": "system", "content": "You are an AI that resolves pronoun references."}
    user_msg = {
        "role": "user",
        "content": f"{example['text']} Who does '{example['pronoun']}' refer to?"
    }
    assistant_msg = {
        "role": "assistant",
        "content": example['options'][example['label']]
    }
    return {"conversations": [system_msg, user_msg, assistant_msg]}

# Apply formatting to each sample
train_dataset = train_dataset.map(format_example)


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [20]:
from unsloth.chat_templates import standardize_sharegpt

# Standardize dataset using Unsloth's chat template
train_dataset = standardize_sharegpt(train_dataset)

Standardizing format:   0%|          | 0/250 [00:00<?, ? examples/s]

In [33]:
import torch
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-3B-Instruct")

# Function to format chat prompts
def formatting_prompts_func(examples):
    # Process each conversation batch
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in examples["conversations"]
    ]
    
    # Tokenize input
    tokenized_output = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Ensure labels are properly converted to tensor format

    label = torch.tensor(examples["label"], dtype=torch.long)  # If already a list
    

    return {
        "input_ids": tokenized_output["input_ids"],
        "attention_mask": tokenized_output["attention_mask"],
        "label": label.unsqueeze(-1)  # Ensure correct shape for loss functions
    }

# Apply tokenizer formatting
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [35]:
print(type(train_dataset[0]["label"]))


<class 'list'>


In [36]:
train_dataset[0]

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.',
 'pronoun': 'they',
 'pronoun_loc': 63,
 'quote': 'they feared violence',
 'quote_loc': 63,
 'options': ['The city councilmen', 'The demonstrators'],
 'label': [0],
 'source': '(Winograd 1972)',
 'conversations': [{'content': 'You are an AI that resolves pronoun references.',
   'role': 'system'},
  {'content': "The city councilmen refused the demonstrators a permit because they feared violence. Who does 'they' refer to?",
   'role': 'user'},
  {'content': 'The city councilmen', 'role': 'assistant'}],
 'input_ids': [128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128004,
  128000,
  128000,
  128006,
  9125,
  128007,
  271,
  38766,
  1303,
  33025,
  2696,
  25,
  6790,
  220,
  2366,
  18,

In [37]:
train_dataset[0]["conversations"]



[{'content': 'You are an AI that resolves pronoun references.',
  'role': 'system'},
 {'content': "The city councilmen refused the demonstrators a permit because they feared violence. Who does 'they' refer to?",
  'role': 'user'},
 {'content': 'The city councilmen', 'role': 'assistant'}]

In [38]:
train_dataset[0]["text"]

'The city councilmen refused the demonstrators a permit because they feared violence.'

In [26]:
import torch
print(torch.__version__)

2.5.1+cu121


In [27]:
pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, cut-cross-entropy, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision, unsloth, unsloth_zoo, xformers
Note: you may need to restart the kernel to use updated packages.


In [39]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

In [40]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=4):   0%|          | 0/250 [00:00<?, ? examples/s]

In [41]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])


"<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Mar 2025\n\nYou are an AI that resolves pronoun references.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe city councilmen refused the demonstrators a permit because they feared violence. Who does 'they' r

In [42]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

'                                                                                             \n\nThe city councilmen<|eot_id|>'

In [43]:
from unsloth import unsloth_train
trainer_stats = unsloth_train(trainer)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 250 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)


AssertionError: 